# Chapter 1
## Querying a Table
```SAS
proc sql;
    select empid,jobcode,salary, salary*.06 as bonus 
    from sasuser.payrollmaster 
    where salary<32000
    order by jobcode;
quit;
```
## Summarizing Groups of Data
```SAS
proc sql;
    select membertype, sum(milestraveled) as TotalMiles 
    from sasuser.frequentflyers
    group by membertype;
quit;
```
## Creating a Table from the Results of a Query on Two Tables
```SAS
proc sql;
    create table work.miles as
    select salcomps.empid, lastname, newsals.salary, newsalary
    from sasuser.salcomps,sasuser.newsals 
    where salcomps.empid=newsals.empid 
    order by 2;
quit;
```

## Displaying All Columns in Output and an Expanded Column List in the SAS Log
```SAS
proc sql feedback; 
    select *
    from sasuser.staffchanges; 
quit;
``` 
## Eliminating Duplicate Rows from Output
```SAS
proc sql;
    select distinct flightnumber, destination
    from sasuser.internationalflights
    order by 1; 
quit;
```
## Subsetting Rows By Using Calculated Values
```SAS
proc sql outobs=10; validate
    select flightnumber, date label="Flight Date", destination, 
            boarded + transferred + nonrevenue as Total
    from sasuser.marchflights
    where calculated total between 100 and 150; 
quit;
```
## Subsetting Data By Using a Noncorrelated Subquery
```SAS
proc sql noexec; 
    select jobcode, avg(salary) as AvgSalary format=dollar11.2 
    from sasuser.payrollmaster 
    group by jobcode
    having avg(salary) > (select avg(salary) from sasuser.payrollmaster);
quit;
```
## Subsetting Data By Using a Correlated Subquery
```SAS
proc sql;
title 'Frequent Flyers Who Are Not Employees'; 
    select count(*) as Count
    from sasuser.frequentflyers 
    where not exists
        (select *
        from sasuser.staffmaster 
        where name = trim(lastname)||', '||firstname);
quit;
```

# Chapter 2
## Combining Tables By Using an Inner Join
```SAS
proc sql outobs=15;
title 'New York Employees';
    select substr(firstname,1,1) || '. ' || lastname as Name, jobcode,
    int((today() - dateofbirth)/365.25) as Age
    from sasuser.payrollmaster as p, sasuser.staffmaster as s
    where p.empid = s.empid and state='NY' 
    order by 2, 3;
quit;
```
## Combining Tables By Using a Left Outer Join
```SAS
proc sql outobs=20;
title 'All March Flights';
    select m.date, m.flightnumber label='Flight Number', 
    m.destination label='Left', f.destination label='Right',
    delay label='Delay in Minutes' 
    from sasuser.marchflights as m left join sasuser.flightdelays as f 
    on m.date=f.date and m.flightnumber= f.flightnumber
    order by delay; 
quit;
```
## Overlaying Common Columns in a Full Outer Join
```SAS
proc sql;
    select coalesce(p.empid, s.empid) as ID, firstname, lastname, gender 
    from sasuser.payrollmaster as p full join sasuser.staffmaster as s
    on p.empid = s.empid
    order by id; 
quit;
```
## Joining Tables By Using a Subquery and an In-Line View
```SAS
proc sql; 
    select empid
    from sasuser.supervisors as m,
        (select substr(jobcode,1,2) as JobCategory, state
        from sasuser.staffmaster as s, sasuser.payrollmaster as p 
        where s.empid = p.empid and s.empid in
            (select empid
            from sasuser.flightschedule 
            where date='04mar2000'd and destination='CPH')
        ) as c 
    where m.jobcategory = c.jobcategory and m.state = c.state;
quit;
```

# Chapter 3
## Combining Tables By Using an Inner Join
```SAS
proc sql outobs=15;
title 'New York Employees';
    select substr(firstname,1,1) || '. ' || lastname as Name,
    jobcode, int((today() - dateofbirth)/365.25) as Age
    from sasuser.payrollmaster as p, sasuser.staffmaster as s
    where p.empid = s.empid and state='NY' 
    order by 2, 3;
quit;
```
## Combining Tables By Using a Left Outer Join
```SAS
proc sql outobs=20;
title 'All March Flights';
    select m.date, m.flightnumber label='Flight Number', m.destination label='Left', 
           f.destination label='Right', delay label='Delay in Minutes' 
    from sasuser.marchflights as m left join sasuser.flightdelays as f 
    on m.date=f.date and m.flightnumber= f.flightnumber
    order by delay; 
quit;
```
## Overlaying Common Columns in a Full Outer Join
```SAS
proc sql;
    select coalesce(p.empid, s.empid) as ID, firstname, lastname, gender 
    from sasuser.payrollmaster as p full join sasuser.staffmaster as s
    on p.empid = s.empid
    order by id; 
quit;
```
## Joining Tables By Using a Subquery and an In-Line View
```SAS
proc sql; 
    select empid
    from sasuser.supervisors as m,
        (select substr(jobcode,1,2) as JobCategory, state
        from sasuser.staffmaster as s, sasuser.payrollmaster as p 
        where s.empid=p.empid and s.empid in
            (select empid
            from sasuser.flightschedule where date='04mar2000'd
            and destination='CPH')
        ) as c 
    where m.jobcategory=c.jobcategory and m.state=c.state;
quit;
```

# Chapter 4
## Combining Tables Vertically
```SAS
proc sql;
    select firstname, lastname
    from sasuser.staffchanges 
    intersect all
    select firstname, lastname
    from sasuser.staffmaster; 
quit;
```

# Chapter 5
## Creating an Empty Table By Defining Columns
```SAS
proc sql;
    create table work.discount
    (Destination char(3), BeginDate num Format=date9., 
    EndDate num format=date9., Discount num);
quit;
```
## Creating an Empty Table That Is like Another Table
```SAS
proc sql;
    create table work.flightdelays2
    (drop=delaycategory destinationtype) like sasuser.flightdelays;
quit;
```
## Creating a Table from a Query Result
```SAS
proc sql;
    create table work.ticketagents as
    select lastname, firstname, jobcode, salary
    from sasuser.payrollmaster, sasuser.staffmaster 
    where payrollmaster.empid = staffmaster.empid and jobcode contains 'TA';
quit;
```
## Displaying the Structure of a Table
```SAS
proc sql;
    describe table work.discount;
quit;
```
## Inserting Rows into a Table By Specifying Column Names and Values
```SAS
proc sql;
    insert into work.discount
    set destination='LHR', begindate='01MAR2000'd, enddate='05MAR2000'd, discount=.33
    set destination='CPH', begindate='03MAR2000'd, enddate='10MAR2000'd, discount=.15;
quit;
```
## Inserting Rows into a Table By Specifying Lists of Values
```SAS
proc sql;
    insert into work.discount 
    (destination, begindate,enddate, discount) 
    values ('LHR','01MAR2000'd,'05MAR2000'd,.33) 
    values ('CPH','03MAR2000'd,'10MAR2000'd,.15);
quit;
```
## Inserting Rows into a Table from a Query Result
```SAS
proc sql;
    insert into work.payrollchanges2
    select empid, salary, dateofhire
    from sasuser.payrollmaster
    where empid in ('1919','1350','1401');
quit;
```
## Creating a Table That Has Integrity Constraints
```SAS
proc sql;
    create table work.employees
    (Name char(10),
    Gender char(1),
    HDate date label='Hire Date' not null, 
    constraint prim_key primary key(name), 
    constraint gender check(gender in ('M' 'F'))
    );
quit;
```
## Displaying Integrity Constraints for a Table
```SAS
proc sql;
    describe table constraints work.discount4;
quit;
```
## Updating Rows in a Table Based on an Expression
```SAS
proc sql;
    update work.payrollmaster_new
    set salary=salary*1.05
    where jobcode like '__1'; 
quit;
```
## Updating Rows in a Table By Using a CASE Expression
```SAS
proc sql;
    update work.payrollmaster_new
    set salary=salary* 
    case
        when substr(jobcode,3,1)='1' then 1.05
        when substr(jobcode,3,1)='2' then 1.10
        when substr(jobcode,3,1)='3' then 1.15
        else 1.08 
    end;
quit;
```
## Updating Rows in a Table By Using a CASE Expression 2
```SAS
proc sql outobs=10;
    select lastname, firstname, jobcode,
        case substr(jobcode,3,1) when '1' then 'junior' 
             when '2' then 'intermediate' 
             when '3' then 'senior' 
             else 'none'
        end as JobLevel
    from sasuser.payrollmaster, sasuser.staffmaster 
    where staffmaster.empid = payrollmaster.empid;
quit;
```
## Deleting Rows in a Table
```SAS
proc sql;
    delete from work.frequentflyers2
    where pointsearned-pointsused<=0; 
quit;
```
## Adding, Modifying, and Dropping Columns in a Table
```SAS
proc sql;
    alter table work.payrollmaster4
    add Age num
    modify dateofhire date format=mmddyy10. 
    drop dateofbirth, gender;
quit;
```
## Dropping a Table
```SAS
proc sql;
    drop table work.payrollmaster4;
quit;
```

# Chapter 6
## Creating a Simple, Unique Index, and a Composite Index
```SAS
proc sql;
    create unique index EmpID on work.payrollmaster(empid); 
    create index daily on work.marchflights(flightnumber,date); 
quit;
```
## Displaying Index Specifications
```SAS
proc sql;
    describe table marchflights;
quit;
```
## Determining Whether SAS Is Using an Index
```SAS
options msglevel=i; 
proc sql;
    select *
    from marchflights
    where flightnumber='182';
quit;
```
## Directing SAS to Ignore All Indexes
```SAS
proc sql; 
    select *
    from marchflights (idxwhere=no)
    where flightnumber='182'; 
quit;
```
## Directing SAS to Use a Specified Index
```SAS
proc sql; 
    select *
    from marchflights (idxname=daily)
    where flightnumber='182'; 
quit;
```
## Dropping an Index
```SAS
proc sql;
    drop index daily
    from work.marchflights; 
quit;
```

# Chapter 7
## Creating a PROC SQL View
```SAS
proc sql;
    create view sasuser.raisev as
    select empid, jobcode, salary format=dollar12.2, 
           salary/12 as MonthlySalary format=dollar12.
    from payrollmaster
    using libname airline 'c:\data\ia';
quit;
```
## Displaying the Definition for a PROC SQL View
```SAS
proc sql;
    describe view sasuser.raisev;
quit;
```
## Using a PROC SQL View in a Query
```SAS
proc sql; 
    select *
    from sasuser.raisev
    where jobcode in ('PT2','PT3'); 
quit;
```
## Updating a PROC SQL View
```SAS
proc sql;
    update sasuser.raisev
    set salary=salary * 1.20
    where jobcode='PT3'; 
quit;
```
## Dropping a PROC SQL View
```SAS
proc sql;
    drop view sasuser.raisev;
quit;
```

# Chapter 8
## Querying a Table Using PROC SQL Options
```SAS
proc sql outobs=5;
    select flightnumber, destination
    from sasuser.internationalflights; 
reset number;
    select flightnumber, destination 
    from sasuser.internationalflights 
    where boarded gt 200;
quit;
```
## Describing and Querying a Dictionary Table
```SAS
proc sql;
    describe table dictionary.columns; 
    select memname
    from dictionary.columns 
    where libname='SASUSER' and name='EmpID';
quit;
```